<a href="https://colab.research.google.com/github/stawank/AdvancedCppConcepts/blob/main/cuda_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'nvidia-driver-550-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-550-server-550.144.03' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-575-open' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-560-server-560.28.03' for glob 'nvidia*'
Note, selecting 'nvidia-driver-570-server' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-doc' for glob 'nvidia*'
Note, selecting 'nvidia-imex' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-firmware-535-server-535.154.05' for glob 'nvidia*'
Note, selecting 'nvidia-headless-390' for glob 'nvidia*'
Note, selecting 'nvidia-cuda-toolkit-gcc' for glob 'nvidia*'
Note, selecting 'nvidia-headless-418' for glob 'nvidia*'
Note, selecting 'nvidia

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2025-04-29 04:32:47--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 173.223.234.21, 173.223.234.28, 173.223.234.16, ...
Connecting to developer.nvidia.com (developer.nvidia.com)|173.223.234.21|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2025-04-29 04:32:48--  https://developer.nvidia.com/downloads/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?__token__=exp=1745901768~hmac=1d98834530

In [ ]:
!nvidia-smi

Tue Apr 29 04:33:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit g++

Get:1 file:/var/cuda-repo-9-2-local  InRelease
Ign:1 file:/var/cuda-repo-9-2-local  InRelease
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-2-local  Release [574 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: file:/var/cuda-repo-9-2-local/Release.gpg: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-

In [ ]:
%%writefile abc.cu
#include <stdio.h>
#define TILE_WIDTH 16

__global__ void TiledMatrixMultiplication(float* C, float* A, float* B, int numARows, int numAColumns, int numBColumns ) {
    int row = blockIdx.y * TILE_WIDTH + threadIdx.y;
    int col = blockIdx.x * TILE_WIDTH + threadIdx.x;

    float Cvalue = 0.f;

    for (int m = 0; m < (numAColumns + TILE_WIDTH - 1) / TILE_WIDTH; ++m) {
        // Shared memory for tiles of A and B
        __shared__ float As[TILE_WIDTH][TILE_WIDTH];
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

        if (row < numARows && m * TILE_WIDTH + threadIdx.x < numAColumns)
            As[threadIdx.y][threadIdx.x] = A[row * numAColumns + m * TILE_WIDTH + threadIdx.x];
        else
            As[threadIdx.y][threadIdx.x] = 0.0f;

        if (m * TILE_WIDTH + threadIdx.y < numAColumns && col < numBColumns)
            Bs[threadIdx.y][threadIdx.x] = B[(m * TILE_WIDTH + threadIdx.y) * numBColumns + col];
        else
            Bs[threadIdx.y][threadIdx.x] = 0.0f;

        __syncthreads();

        // Multiply the two tiles together
        for (int k = 0; k < TILE_WIDTH; ++k) {
            Cvalue += As[threadIdx.y][k] * Bs[k][threadIdx.x];
        }
        __syncthreads();
    }
    if (row < numARows && col < numBColumns)
        C[row * numBColumns + col] = Cvalue;
}

int main() {
    int numARows = 1024, numAColumns = 1024;
    int numBRows = numAColumns, numBColumns = 1024;
    int numCRows = numARows, numCColumns = numBColumns;

    int sizeA = numARows * numAColumns * sizeof(float);
    int sizeB = numBRows * numBColumns * sizeof(float);
    int sizeC = numCRows * numCColumns * sizeof(float);

    // Allocate host memory
    float *h_A = (float*) malloc(sizeA);
    float *h_B = (float*) malloc(sizeB);
    float *h_C = (float*) malloc(sizeC);

    // Initialize matrices A and B (for example, fill with 1.0f)
    for (int i = 0; i < numARows * numAColumns; i++) {
        h_A[i] = 1.f;
    }
    for (int i = 0; i < numBRows * numBColumns; i++) {
        h_B[i] = 1.f;
    }


    float *d_A, *d_B, *d_C;
    cudaMalloc((void**)&d_A, sizeA);
    cudaMalloc((void**)&d_B, sizeB);
    cudaMalloc((void**)&d_C, sizeC);

    // Define block and grid dimensions
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH);
    dim3 dimGrid((numCColumns + TILE_WIDTH - 1) / TILE_WIDTH, (numCRows + TILE_WIDTH - 1) / TILE_WIDTH);

    // Launch the tiled matrix multiplication kernel
    TiledMatrixMultiplication<<<dimGrid, dimBlock>>>(d_C, d_A, d_B, numARows, numAColumns, numBColumns);
    cudaDeviceSynchronize();

    // Copy the result matrix back to host memory
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);

    for (int i = 0; i < numCRows * numCColumns; i++) {
        if (fabs(h_C[i] - numAColumns) > 1e-5) {
            printf("Error at element %d: %f (expected %d)\n", i, h_C[i], numAColumns);
            break;
        }
    }
    printf("Tiled matrix multiplication completed successfully.\n");

    // Free device and host memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    free(h_A);
    free(h_B);
    free(h_C);

    return 0;
}

Writing abc.cu


In [ ]:
!nvcc abc.cu -o abc

In [ ]:
!./abc

Error at element 0: 0.000000 (expected 1024)
Tiled matrix multiplication completed successfully.


In [ ]:
!nvprof ./abc

==380== NVPROF is profiling process 380, command: ./abc
Error at element 0: 0.000000 (expected 1024)
Tiled matrix multiplication completed successfully.
==380== Profiling application: ./abc
==380== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  1.9664ms         1  1.9664ms  1.9664ms  1.9664ms  [CUDA memcpy DtoH]
      API calls:   93.12%  200.85ms         3  66.951ms  119.63us  200.61ms  cudaMalloc
                    4.78%  10.309ms         1  10.309ms  10.309ms  10.309ms  cudaLaunchKernel
                    1.70%  3.6768ms         1  3.6768ms  3.6768ms  3.6768ms  cudaMemcpy
                    0.30%  647.71us         3  215.90us  167.34us  242.49us  cudaFree
                    0.08%  169.12us       114  1.4830us     104ns  70.949us  cuDeviceGetAttribute
                    0.01%  11.419us         1  11.419us  11.419us  11.419us  cuDeviceGetName
                    0.00%  9.6670us         1  9.6670us  9.

In [ ]:
%%writefile matrix_mul.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Error checking macro
#define cudaCheckError(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
    if (code != cudaSuccess)
    {
        fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
        if (abort) exit(code);
    }
}

// CUDA kernel for matrix multiplication
__global__ void matrixMulKernel(float* A, float* B, float* C, int width)
{
    // Calculate the row and column index of the element
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < width && col < width) {
        float sum = 0.0f;
        // Each thread computes one element of C
        for (int i = 0; i < width; i++) {
            sum += A[row * width + i] * B[i * width + col];
        }
        C[row * width + col] = sum;
    }
}

// Host code
class CUDAMatrix {
private:
    float* h_data;  // Host data
    float* d_data;  // Device data
    int width;
    int size;

public:
    CUDAMatrix(int w) : width(w) {
        size = width * width * sizeof(float);

        // Allocate host memory
        h_data = new float[width * width];

        // Allocate device memory
        cudaCheckError(cudaMalloc(&d_data, size));
    }

    ~CUDAMatrix() {
        if (h_data) delete[] h_data;
        if (d_data) cudaFree(d_data);
    }

    // Initialize matrix with some values
    void initialize() {
        for (int i = 0; i < width * width; i++) {
            h_data[i] = rand() / (float)RAND_MAX;
        }
        // Copy to device
        cudaCheckError(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));
    }

    float* getDeviceData() { return d_data; }
    float* getHostData() { return h_data; }

    // Copy data back to host
    void copyToHost() {
        cudaCheckError(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));
    }

    // Print matrix (first few elements)
    void print(const char* name) {
        printf("\nMatrix %s (showing first 4x4 elements):\n", name);
        for (int i = 0; i < std::min(4, width); i++) {
            for (int j = 0; j < std::min(4, width); j++) {
                printf("%.2f ", h_data[i * width + j]);
            }
            printf("\n");
        }
    }
};

int main() {
    // Matrix dimensions
    int width = 1024;  // 1024x1024 matrices

    // Create matrices
    CUDAMatrix A(width);
    CUDAMatrix B(width);
    CUDAMatrix C(width);

    // Initialize input matrices
    A.initialize();
    B.initialize();

    // Set up execution configuration
    dim3 threadsPerBlock(16, 16);  // 16x16 = 256 threads per block
    dim3 blocksPerGrid((width + threadsPerBlock.x - 1) / threadsPerBlock.x,
                       (width + threadsPerBlock.y - 1) / threadsPerBlock.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Record start time
    cudaEventRecord(start);

    // Launch kernel
    matrixMulKernel<<<blocksPerGrid, threadsPerBlock>>>(
        A.getDeviceData(),
        B.getDeviceData(),
        C.getDeviceData(),
        width
    );

    // Record stop time
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    // Check for kernel launch errors
    cudaCheckError(cudaGetLastError());

    // Calculate elapsed time
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // Copy result back to host
    C.copyToHost();

    // Print results
    A.print("A");
    B.print("B");
    C.print("C");

    printf("\nMatrix multiplication completed in %.2f ms\n", milliseconds);

    // Cleanup CUDA events
    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}

Writing matrix_mul.cu


In [ ]:
!nvcc -O3 matrix_mul.cu -o matrix_mul

In [ ]:
!./matrix_mul


Matrix A (showing first 4x4 elements):
0.84 0.39 0.78 0.80 
0.18 0.36 0.44 0.85 
0.13 0.02 0.63 0.80 
0.77 0.75 0.83 0.75 

Matrix B (showing first 4x4 elements):
0.38 0.45 0.18 0.53 
0.63 0.32 0.74 0.12 
0.02 0.95 0.19 0.60 
0.92 0.65 0.87 0.40 

Matrix C (showing first 4x4 elements):
264.16 257.26 250.82 261.54 
256.46 251.64 252.99 252.48 
261.32 257.26 249.17 251.77 
260.24 259.32 251.85 249.62 

Matrix multiplication completed in 9.11 ms


In [ ]:
!pip install nvcc4jupyter
# to run cu code on notebook only

In [ ]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp91qf5y2d".


In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values
c = 6;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 0

